experiment: first, train it on my posts
make a language one first

then.. hmm, not sure?

maybe make a small one that can play a game at super low res? and RL it?

have one that determines reward/punishment and one that decides actions?
not sure lol but i want to win at panda bomber

In [2]:
import jax
import jax.numpy as jnp
import jax.random as jrand
import optax
import functools # for partial


dataset_path = "../data/shakespeare.txt"
with open(dataset_path, 'r') as f:
  dataset_chars = f.read()


vocab = list(set(dataset_chars))
vocab_size = len(vocab)



token_to_char_dict = dict(enumerate(vocab))
char_to_token_dict = (dict([(b, a) for a, b in token_to_char_dict.items()]))

encode = lambda word: jnp.array([char_to_token_dict[c] for c in word])
decode = lambda tokens: "".join([token_to_char_dict[t] for t in tokens.tolist()])

test_word = "lmaoooo"
print(test_word, encode(test_word), decode(encode(test_word)))


dataset = encode(dataset_chars)
split = 0.9
dataset_token_count = len(dataset)
split_idx = int(dataset_token_count*0.9)

train_data = dataset[:split_idx]
test_data = dataset[split_idx:]


def init_model_params(key, attention_blocks, model_dim, kq_dim, forward_mlp_hidden_layers, embedding_table_hidden_layers, vocab_size, output_mlp_hidden_layers, attention_heads):
  # inits model params
  # function body logic:
  #   generate params for the main transformer blocks
  #   the embed table
  #   and the final output MLP

  key_types = 3 # (+ 3) + len(embedding_table_hidden_layers)*2)*100
  keys = jrand.split(key, key_types)
  # weight initialization functions
  # he for sigmoid, xavier for tanh
  he = lambda rkey, shape : jrand.normal(rkey, shape) * jnp.sqrt(2.0 / shape[0])
  xavier = lambda rkey, shape : jrand.normal(rkey, shape) * jnp.sqrt(2.0 / (shape[0] + shape[1]))

  # mlp takes an input embedding and outputs an embedding.
  mlp_all_layer_sizes = jax.lax.concatenate([
    jnp.array([model_dim]),
    jnp.array(forward_mlp_hidden_layers),
    jnp.array([model_dim])],
    0
  )
  mlp_all_layer_shapes = [(a, b) for a, b in zip(mlp_all_layer_sizes, mlp_all_layer_sizes[1:])]

  d_q = kq_dim
  d_k = d_q
  d_v = kq_dim # for now

  kqv_key_count = attention_blocks*attention_heads*4
  forward_mlp_key_count = len(forward_mlp_hidden_layers)*2
  attention_block_keys = jrand.split(keys[0], kqv_key_count + forward_mlp_key_count)
  model_params = [
    {
      "attention_params" : [
        {
          "Wq" : xavier(attention_block_keys[attention_block*attention_head + 0], (model_dim, d_q//attention_heads)), # B, T, C => B, T, q
          "Wk" : xavier(attention_block_keys[attention_block*attention_head + 1], (model_dim, d_k//attention_heads)), # B, T, C => B, T, k
          "Wv" : xavier(attention_block_keys[attention_block*attention_head + 2], (model_dim, d_v//attention_heads)), # B, T, C => B, T, V.        B, T, T, @ B, T, V => B, T, V
          "Wout" : xavier(attention_block_keys[attention_block*attention_head + 2], (d_v//attention_heads, model_dim//attention_heads)), # B, T, V => B, T, C  ## i could just have one maybe?
        }
        for attention_head in range(attention_heads)
      ],
      "forward_mlp_params" : [
        {
          "W" : xavier(attention_block_keys[attention_block + layer], shape),
          "b" : he(attention_block_keys[attention_block + layer + 1], (shape[1],)),
        }
        for layer, shape in enumerate(mlp_all_layer_shapes)
      ],
    }
    for attention_block in range(attention_blocks)
  ]

  # embed table params
  embed_table_keys = jrand.split(keys[1], 2*len(embedding_table_hidden_layers))
  embed_table_all_layer_sizes = jax.lax.concatenate([
    jnp.array([vocab_size]),
    jnp.array(embedding_table_hidden_layers),
    jnp.array([model_dim])], 0)
  embed_table_all_layer_shapes = [(a, b) for a, b in zip(embed_table_all_layer_sizes, embed_table_all_layer_sizes[1:])]
  embedding_table_params = [
        {
          "W" : xavier(embed_table_keys[layer + 0], shape),
          "b" : he(embed_table_keys[layer + 1], (shape[1],)),
        }
        for layer, shape in enumerate(embed_table_all_layer_shapes)
  ]

  model_params[0].update({
    "embedding_table_params" : embedding_table_params
  })

  output_mlp_all_layer_sizes = jax.lax.concatenate([
    jnp.array([model_dim]),
    jnp.array(output_mlp_hidden_layers),
    jnp.array([vocab_size])
  ], 0)

  # output MLP params. this takes the contextual embeddings and turns them into logits
  output_mlp_keys = jrand.split(keys[2], 2*len(output_mlp_hidden_layers))
  output_mlp_all_layer_shapes = [(a, b) for a, b in zip(output_mlp_all_layer_sizes, output_mlp_all_layer_sizes[1:])]
  output_mlp_params = [
    {
      "W" : xavier(output_mlp_keys[layer + 0], shape),
      "b" : he(output_mlp_keys[layer + 1], (shape[1],))
    }
    for layer, shape in enumerate(output_mlp_all_layer_shapes)
  ]
  model_params[-1].update({
    "output_mlp_params" : output_mlp_params
  })

  return model_params



# for both the mlp in attention blocks and the final output mlp.
# in fact... i could probably use this for the embed table too. lmao
@jax.jit
def mlp_forward(mlp_params, x_input):
  #linear = lambda x, params: (lambda x, W, b: x @ W + b)(x, params["W"], params["b"])

  # doesnt work if leading axis sizes are different for some reason :(
  #return jax.lax.scan(linear, contextual_embeddings, attention_block_params["forward"])

  x = x_input
  for layer_params in mlp_params:
    x = jax.nn.tanh(x @ layer_params["W"] + layer_params["b"]) # tanh : -1, 1
  
  return x


@functools.partial(jax.jit, static_argnames=["vocab_size"])
def embed_tokens(embedding_table_params, xBT, vocab_size=vocab_size):
  xBTC = jax.nn.one_hot(xBT, num_classes=vocab_size)
  return mlp_forward(embedding_table_params, xBTC)



@jax.jit
def attention(attention_params, d_k, xBTC):
  # iterate through each attention head
  # inefficient but its a start. may optimize later.
  for i, attention_head in enumerate(attention_params):
    Q = xBTC @ attention_head["Wq"] # (B, T, C) @ (C, kq_dim) => (B, T, kq_dim)
    K = xBTC @ attention_head["Wk"] # (B, T, C) @ (C, kq_dim) => (B, T, kq_dim)
    V = xBTC @ attention_head["Wv"] # (B, T, C) @ (C, d_v) => (B, T, d_v)
    attention_table = jax.nn.softmax((Q @ jnp.transpose(K, (0, 2, 1)))/d_k) # (B, T, kq_dim) @ (B, kq_dim, T) => (B, T, T)

    Z = attention_table @ V  #  (B, T, T) @ (B, T, d_v) => (B, T, d_v)
    contextual_update_fragment = Z @ attention_head["Wout"] # (B, T, d_v) @ (d_v, C) => (B, T, C)

    # concat attention heads
    if i == 0:
      contextual_update = contextual_update_fragment
    else:
      contextual_update = jax.lax.concatenate([contextual_update, contextual_update_fragment], 2)

  return contextual_update

def norm(channel):
  epsilon = 1e-7
  return (channel - jnp.mean(channel)) / jnp.sqrt(jnp.var(channel) + epsilon)


def layer_norm(BTC):
  # norm over C
  # var = σ^2
  # we want to divide by the standard deviation
  return jax.vmap(norm, in_axes=-1, out_axes=-1)(BTC)


### forward
@jax.jit
def forward(model_params, xBTC, d_k):
  for attention_block in range(len(model_params)):
    ## attention
    contextual_embeddings = attention(
      model_params[attention_block]["attention_params"],
      d_k,
      xBTC
    ) # B, T, C

    # skip connection or whatever
    mlp_out = mlp_forward(model_params[attention_block]["forward_mlp_params"], contextual_embeddings)
    residual = layer_norm(mlp_out)
    xBTC = xBTC + residual

  # finally, do something with the output
  # doesnt matter what. predict the next token. take the next action. lol its whatever.
  # for now just do next token prediction. implement a normal transformer
  xBTC = mlp_forward(model_params[-1]["output_mlp_params"], xBTC)

  return xBTC


@jax.jit
def inference(key, model_params, xBT, d_k, temperature=0.5):
  embedding_table_params = model_params[0]["embedding_table_params"]
  xBTC = embed_tokens(embedding_table_params, xBT)
  yBTC = forward(model_params, xBTC, d_k)
  next_token_logits = yBTC[0][-1] # the first B and last T in (B, T, C)
  #next_token_probs = jax.nn.softmax(next_token_logits, axis=-1) # (C,)
  # argmax for display
  probs = jax.nn.softmax(next_token_logits / (0.01 + temperature))
  a = jnp.arange(probs.shape[-1])
  return jrand.choice(key, a, replace=True, p=probs, axis=-1)


@jax.jit
def loss_func(model_params, xBT, yB, d_k):
  embedding_table_params = model_params[0]["embedding_table_params"]
  xBTC = embed_tokens(embedding_table_params, xBT)
  yBTC = forward(model_params, xBTC, d_k)
  next_token_logits = yBTC[:, -1, :] # the last T in (B, T, C)
  # honestly this is stupid!!! C is a list of attributes of a token, 'logits' is the wrong datatype, even if theyre both represented as float
  # fuck it just do baseline for now
  next_token_probs = jax.nn.softmax(next_token_logits, axis=-1) # (B, C)
  true_token_probs = jax.nn.one_hot(yB, next_token_probs.shape[-1], axis=-1) # (B, C)
  batch_crossentropy = -true_token_probs * jax.nn.log_softmax(next_token_probs)
  return jnp.mean(batch_crossentropy)


# static args optimizer
@functools.partial(jax.jit, static_argnames=["optimizer"])
def train_step(model_params, xBT, yB, d_k, opt_state, optimizer):
  loss, grads = jax.value_and_grad(loss_func)(model_params, xBT, yB, d_k)
  param_updates, updated_opt_state = optimizer.update(grads, opt_state, model_params)
  updated_model_params = optax.apply_updates(model_params, param_updates)
  return updated_model_params, updated_opt_state, loss, grads



# model params
attention_blocks = 2
attention_heads = 8
model_dim = 512
kq_dim = 1024
d_k = model_dim // attention_heads

forward_mlp_hidden_layers = [
  1024,
  1024,
  1024,
]

output_mlp_hidden_layers = [
  1024,
  1024,
  1024,
]

embedding_table_hidden_layers = [
  1024,
  1024,
  1024
]


# set up training
key = jrand.PRNGKey(112367)
model_params = init_model_params(key, attention_blocks, model_dim, kq_dim, forward_mlp_hidden_layers, embedding_table_hidden_layers, vocab_size, output_mlp_hidden_layers, attention_heads)

lr = 3e-4 # karpathy style
optimizer = optax.adamw(learning_rate=lr)
opt_state = optimizer.init(model_params)


epochs = 1000
train_batch_size = 100
val_batch_size = 100
max_sequence_length = 29 + 1 # xlen + ylen

tokens_per_step = train_batch_size*max_sequence_length




# todo masking

lmaoooo [17 59 29  3  3  3  3] lmaoooo


In [14]:
for epoch in range(epochs):
  steps = len(train_data) // (tokens_per_step) # idk calculate train_dataset_size // batch_size
  for step in range(steps):
    test_tokens = train_data[step*tokens_per_step:(step+1)*tokens_per_step].reshape(-1, max_sequence_length) # (B, T) where T = max seq len
    xBT = test_tokens[:, :-1] # get all but last token in each sequence
    # i need to turn this into every preceding subsequence. i.e. "abcd" => (xss, ys) => ([a, ab, abc], [b, c, d])
    yB = test_tokens[:, -1] # lol ez
    inference_key = jrand.PRNGKey(epoch * step + step)
    yhat = inference(inference_key, model_params, xBT, d_k)
    model_params, opt_state, loss, grads = train_step(model_params, xBT, yB, d_k, opt_state, optimizer)

    in_chars = decode(xBT[0]).replace('\n', '↵')
    target_char = decode(jnp.array([yB[0]]))
    pred_char = decode(jnp.array([yhat]))

    print(f"{epoch}, {step}, {loss}, '{in_chars}' => '{pred_char}' =?> '{target_char}'")


0, 0, 0.06378274410963058, 'First Citizen:↵Before we proc' => 'n' =?> 'e'
0, 1, 0.0638112798333168, 'ever↵Appear in your impedimen' => 'v' =?> 't'
0, 2, 0.06376562267541885, 'eart, to the seat o' the brai' => 'n' =?> 'n'
0, 3, 0.0638321116566658, 'usands of these quarter'd sla' => 'r' =?> 'v'
0, 4, 0.06387902796268463, 'eads on at noon: but I do won' => 's' =?> 'd'
0, 5, 0.06390633434057236, 'ould not↵sell him an hour fro' => 't' =?> 'm'
0, 6, 0.06394306570291519, '↵↵VIRGILIA:↵Indeed, no, by yo' => 'a' =?> 'u'
0, 7, 0.06381576508283615, 'ow put your shields before yo' => 'r' =?> 'u'
0, 8, 0.0637977123260498, ':↵Thy friend no less↵Than tho' => 'o' =?> 's'
0, 9, 0.0638524666428566, 'de to endure friends, that yo' => 'e' =?> 'u'
0, 10, 0.06382378935813904, ' charter to extol her blood,↵' => ' ' =?> 'W'
0, 11, 0.06383182108402252, 'is yours. What is't?↵↵CORIOLA' => 'j' =?> 'N'
0, 12, 0.06386390328407288, 'do you?↵↵Both:↵Why, how are w' => 'i' =?> 'e'
0, 13, 0.06388010084629059, 'ging of yo

KeyboardInterrupt: 

In [15]:
def get_completion(key, prompt, d_k=d_k):
  # this is LSTM style, not true transformer style.
  xBT = encode(prompt)[None, :] # fake batch
  yhat = inference(key, model_params, xBT, d_k) # will be messed up bc argmax instead of choosing probs
  return decode(jnp.array([yhat]))

import time
key = jrand.PRNGKey(int(time.time()*100))

prompt = "First Citizen:\nBefore we proc"
print(prompt, end='')
for i in range(100):
  next_char = get_completion(key, prompt[-max_sequence_length:])
  prompt += next_char
  print(next_char, end='')

First Citizen:
Before we procaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [9]:
print(xBT.shape, yB.shape)
jax.tree_util.tree_map(jnp.linalg.norm, grads)


(100, 29) (100,)


[{'attention_params': [{'Wk': Array(5.439577e-09, dtype=float32),
    'Wout': Array(1.7552958e-08, dtype=float32),
    'Wq': Array(6.0527805e-09, dtype=float32),
    'Wv': Array(2.0876437e-08, dtype=float32)},
   {'Wk': Array(1.5989373e-09, dtype=float32),
    'Wout': Array(1.8074301e-08, dtype=float32),
    'Wq': Array(1.6908579e-09, dtype=float32),
    'Wv': Array(1.9549228e-08, dtype=float32)},
   {'Wk': Array(3.5835377e-09, dtype=float32),
    'Wout': Array(1.9075788e-08, dtype=float32),
    'Wq': Array(3.8867554e-09, dtype=float32),
    'Wv': Array(1.7926924e-08, dtype=float32)},
   {'Wk': Array(1.5083843e-09, dtype=float32),
    'Wout': Array(1.5769741e-08, dtype=float32),
    'Wq': Array(2.5607705e-09, dtype=float32),
    'Wv': Array(1.6212404e-08, dtype=float32)},
   {'Wk': Array(3.6827823e-09, dtype=float32),
    'Wout': Array(1.5018214e-08, dtype=float32),
    'Wq': Array(4.1164774e-09, dtype=float32),
    'Wv': Array(1.6350812e-08, dtype=float32)},
   {'Wk': Array(8.429604e-